In [1]:

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
import cv2
import PIL
import math


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device="cpu"
class Config:
    def __init__(self):
        self.epochs = 20
        self.cuda=True
        self.num_classes = 1
        self.batch_size = 4
        self.learning_rate = 0.01
        self.train_size=0.8
        self.dataset = "InterferometerPhoto"
        #self.architecture = "CNN"
        self.pin_memory = True
        self.momentum = 0.9
        self.step_size = 3
        self.gamma = 0.1
        self.dataset_metadata = "../data/raw/1channel/reference/epsilon.csv" # will change for processed
        self.num_workers = 0
        self.data_root_dir = "../data/raw/1channel/photo" # will change for processed
        self.data_transforms = transforms.Compose([
                #transforms.CenterCrop(448),
                #transforms.Resize(224),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.491, 0.491, 0.491],
                                      std=[0.210, 0.210, 0.210]) 
                #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])

class EpsilonDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.annotations = pd.read_csv(annotation_file,skiprows=0, delim_whitespace=' ')
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img = np.array(cv2.imread(os.path.join(self.root_dir, str(str("%05d" %self.annotations.imgnr[index]))+ ".png"))).astype(np.float32)
        img=PIL.Image.fromarray(np.uint8(img))
        y_label = self.annotations.eps[index]
        if self.transform is not None:
            img = self.transform(img)
        return img, y_label

def prepare_data(config):
    dataset = EpsilonDataset(config.data_root_dir, config.dataset_metadata, transform=config.data_transforms)
    
    g = torch.Generator(device=device).manual_seed(0)
    train_size = int(config.train_size * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size], generator=g)
    print("len(train_dataset):", len(train_dataset),"len(val_dataset):", len(val_dataset))
    print(train_dataset)
    loader_params = dict(batch_size=config.batch_size, num_workers=config.num_workers,
                         pin_memory=config.pin_memory, generator=g, shuffle=True)

    train_loader = torch.utils.data.DataLoader(**loader_params, dataset=train_dataset )
    validation_loader = torch.utils.data.DataLoader(**loader_params, dataset=val_dataset )
    #validation_loader = torch.utils.data.DataLoader(**loader_params, dataset=train_dataset )
    return {'train': train_loader, 'val': validation_loader}
    



Load Data

In [3]:
config=Config()

dataloaders = prepare_data(config)
dataset_sizes = {x: len(dataloaders[x]) for x in ['train', 'val']}
#class_names = image_datasets['train'].classes
dataset = EpsilonDataset(config.data_root_dir, config.dataset_metadata, transform=config.data_transforms)
#print(dataset[0])
train_features, train_labels=next(iter(dataloaders["train"]))
#print(f"Feature batch shape: {train_features.size()}")
#print(f"Labels batch shape: {train_labels.size()}")

print("Device: ", device)
print("Dataloader train len: ", len(dataloaders["train"]), "val len: ", len(dataloaders["val"]))

len(train_dataset): 1280 len(val_dataset): 320
Device:  cpu
Dataloader train len:  320 val len:  80


In [4]:
def customLossFunctionDebug(outputs, labels, totalLoss):
    print("NextOne")
    for i in range (len(outputs)):
        print("i: ", i, "label: ", float(labels[i]), "output:", float(outputs[i]), "diff= ", float(min( abs(abs(labels[i])-abs(outputs[i])) , abs(1-(abs(labels[i])-abs(outputs[i]))) )))#float(labels[i])-float(outputs[i]))

    print("totalLoss:", float(totalLoss))
    return totalLoss


def customLossFunction(outputs, labels):
    totalLoss=0.0
    for i in range (len(outputs)):
        #oneOutputLoss=min(abs(abs(labels[i]%1-outputs[i]%1)), 1-(labels[i]%1-outputs[i]%1) )
        oneOutputLoss=min( abs(abs(labels[i])-abs(outputs[i])) , abs(1-(abs(labels[i])-abs(outputs[i]))) )
        totalLoss+=oneOutputLoss
    totalLoss/=len(outputs)
    #customLossFunctionDebug(outputs=outputs, labels=labels, totalLoss=totalLoss)
    return totalLoss

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = -100000
    
    for epoch in range(num_epochs):
        
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        #print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            
            running_loss = 0.0
            running_corrects = 0
            #i=0
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                #print(i, len(inputs))
                #i=i+1
                # zero the parameter gradients
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    #_, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss+=loss
                #running_loss += loss.item() * inputs.size(0)
                #running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            #print("epoch_loss: ", epoch_loss, "running_loss: ", running_loss, "dataset_sizes[phase]: ", dataset_sizes[phase])
            epoch_acc = 1-epoch_loss#running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                str(phase), float(epoch_loss), float(epoch_acc)))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(float(best_acc)))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 1)

model_ft = model_ft.to(device)

# Observe that all parameters are being optimized
#optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)


# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
#print((dataloaders['train'])[0])
model_ft = train_model(model=model_ft, criterion=customLossFunction, optimizer=optimizer_ft, scheduler=exp_lr_scheduler,
                       num_epochs=125)

Epoch 0/124
train Loss: 0.4583 Acc: 0.5417
val Loss: 0.2248 Acc: 0.7752

Epoch 1/124
train Loss: 0.3434 Acc: 0.6566
val Loss: 0.1644 Acc: 0.8356

Epoch 2/124
train Loss: 0.2644 Acc: 0.7356
val Loss: 0.2598 Acc: 0.7402

Epoch 3/124
train Loss: 0.2269 Acc: 0.7731
val Loss: 0.3474 Acc: 0.6526

Epoch 4/124
train Loss: 0.2098 Acc: 0.7902
val Loss: 0.1928 Acc: 0.8072

Epoch 5/124
train Loss: 0.2114 Acc: 0.7886
val Loss: 0.1685 Acc: 0.8315

Epoch 6/124
train Loss: 0.1908 Acc: 0.8092
val Loss: 0.1703 Acc: 0.8297

Epoch 7/124
train Loss: 0.1130 Acc: 0.8870
val Loss: 0.0690 Acc: 0.9310

Epoch 8/124
train Loss: 0.1159 Acc: 0.8841
val Loss: 0.2441 Acc: 0.7559

Epoch 9/124
train Loss: 0.0968 Acc: 0.9032
val Loss: 0.0356 Acc: 0.9644

Epoch 10/124
train Loss: 0.0860 Acc: 0.9140
val Loss: 0.0847 Acc: 0.9153

Epoch 11/124
train Loss: 0.0822 Acc: 0.9178
val Loss: 0.0518 Acc: 0.9482

Epoch 12/124
train Loss: 0.0733 Acc: 0.9267
val Loss: 0.0344 Acc: 0.9656

Epoch 13/124
train Loss: 0.0681 Acc: 0.9319
val 

In [ ]:
torch.save(model_ft.state_dict(), 'FirstModelSaved.pht')

ConvNet as fixed feature extractor
----------------------------------

Here, we need to freeze all the network except the final layer. We need
to set ``requires_grad = False`` to freeze the parameters so that the
gradients are not computed in ``backward()``.

You can read more about this in the documentation
`here <https://pytorch.org/docs/notes/autograd.html#excluding-subgraphs-from-backward>`__.




In [ ]:
model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

Train and evaluate
^^^^^^^^^^^^^^^^^^

On CPU this will take about half the time compared to previous scenario.
This is expected as gradients don't need to be computed for most of the
network. However, forward does need to be computed.




In [ ]:
model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=25)

In [ ]:
visualize_model(model_conv)

plt.ioff()
plt.show()

Further Learning
-----------------

If you would like to learn more about the applications of transfer learning,
checkout our `Quantized Transfer Learning for Computer Vision Tutorial <https://pytorch.org/tutorials/intermediate/quantized_transfer_learning_tutorial.html>`_.



